
<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [2]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [3]:

print("load training set..")
print("\n")
train_data, vocab = load_data("/content/drive/MyDrive/AMMI SLIDES/NLP_Week_1_Labs_2022/session3/train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, vocab_dict= load_data("/content/drive/MyDrive/AMMI SLIDES/NLP_Week_1_Labs_2022/session3/valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [4]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []

    for sentence in data:
      words = []
      for word in sentence:
        if (vocab[word]<=mincount) or (word not in vocab.keys()):
          words.append('<unk>')
        else:
          words.append(word)
      data_with_unk.append(words)

    ## FILL CODE
    
    return data_with_unk

In [5]:
print("remove rare words")

train_data= remove_rare_words(train_data, vocab, mincount = 1)
valid_data= remove_rare_words(valid_data, vocab, mincount = 1)
# train_data

remove rare words


In [6]:
train_data[0]

['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']

In [7]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    word_counts= defaultdict(lambda: 0.0)
    word_in_context=defaultdict(lambda: 0.0)


    for sentence in data:       
      sentence = tuple(sentence)  
      ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
      
      for i in range(len(sentence)):
        for j in range(n):
          if i-j>=0:
            context=sentence[i-j:i]
            counts[context][sentence[i]]+=1
            word_counts[sentence[i]]+=1
            word_in_context[context]+=1

      
    proba  = defaultdict(lambda: defaultdict(lambda: 0.0))
    for context in counts.keys():
        for word in counts[context].keys():
          proba[context][word]=(counts[context][word])/(word_in_context[context])
    # ## FILL CODE
      
  
    return proba


In [8]:
# RUN TO BUILD NGRAM MODEL

n = 3
print("build ngram model with n = ", n)
model= build_ngram(train_data, n)


build ngram model with n =  3


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [9]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Returns:
    prob(float): probability of this word given the context 
    '''
    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE
    if context in model and w in model[context]:
      return model[context][w]
    else:
      return 0.4*get_prob(model, context[1:], w)

In [10]:
import numpy as np
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    pp=0.0
    T=0
    for sentence in data:
      sentence=tuple(sentence)
      T+=len(sentence)-1
      for i in range(1,len(sentence)):
        if n<i+1:
          context=sentence[i+1-n:i]
        else:
          context=sentence[0:i]
        pp-=np.log(get_prob(model,context,sentence[i]))
    perplexity=np.exp(pp/T)
    return perplexity

In [11]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 62.065880784180806


In [12]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    if context in model.keys():
      return model[context]
    else:
      return get_proba_distrib(model,context[1:])
    
    ## FILL CODE
    
    # return None

In [13]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    n=2
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
      sentence_=tuple(sentence)
      if n-len(sentence_)<=0:
        context=sentence_
      else:
        context=sentence_[-n:]
      prob=get_proba_distrib(model,context)
      higehest_prob=max(prob.values())
      list_val=list(prob.values()).index(higehest_prob)
      list_key=list(prob.keys())[list_val]
      sentence.append(list_key)
        ## FILL CODE
        
    return sentence

In [14]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'tom', 'and', 'mary', 'said', 'they', 'were', '<unk>', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.